In [1]:
import transformers
import evaluate
import torch
import operator
import ast
import json
import re
import sys

import numpy as np
import pandas as pd
import torch.nn as nn

from multiprocessing import cpu_count
from evaluate import load
from nusacrowd import NusantaraConfigHelper
from torch.utils.data import DataLoader
from datetime import datetime
from huggingface_hub import notebook_login
from tqdm import tqdm

from datasets import (
    load_dataset, 
    load_from_disk,
    Dataset,
    DatasetDict
)
from transformers import (
    BigBirdTokenizerFast,
    BigBirdForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
    BertForSequenceClassification,
    BertForQuestionAnswering,
    AutoModel, 
    BertTokenizerFast,
    AutoTokenizer, 
    AutoModel, 
    BertTokenizer, 
    BertForPreTraining,
    AutoModelForSequenceClassification,
    AutoModelForQuestionAnswering,
    XLMRobertaTokenizer,
    pipeline
)

In [2]:
def question_type(data, nama):
    num_apa = 0
    num_dimana = 0
    num_kapan = 0
    num_siapa = 0
    num_bagaimana = 0
    num_kenapa = 0
    num_berapa = 0
    num_others = 0
    
    for i in data['question']:
        current_question = i.split()
        
        if 'Apa' in current_question: num_apa += 1
        elif 'Apakah' in current_question:num_apa += 1
        elif 'apa' in current_question:num_apa += 1
        elif 'apakah' in current_question:num_apa += 1
        
        elif 'Dimana' in current_question: num_dimana += 1
        elif 'dimana' in current_question: num_dimana += 1
        elif 'mana' in current_question: num_dimana += 1
        
        elif 'Kapan' in current_question: num_kapan += 1
        elif 'kapan' in current_question: num_kapan += 1
        
        elif 'Siapa' in current_question: num_siapa += 1
        elif 'siapa' in current_question: num_siapa += 1
        
        elif 'Bagaimana' in current_question: num_bagaimana += 1
        elif 'bagaimana' in current_question: num_bagaimana += 1

        elif 'Mengapa' in current_question: num_kenapa += 1
        elif 'Kenapa' in current_question: num_kenapa += 1
        elif 'mengapa' in current_question: num_kenapa += 1
        elif 'kenapa' in current_question: num_kenapa += 1
        
        elif 'Berapa' in current_question: num_berapa += 1
        elif 'Berapakah' in current_question: num_berapa += 1
        elif 'berapa' in current_question: num_berapa += 1
        elif 'berapakah' in current_question: num_berapa += 1
        
        else: num_others += 1
        
    denominator = len(data['question'])
    num_apa_percentage = num_apa / denominator
    num_dimana_percentage = num_dimana / denominator
    num_kapan_percentage = num_kapan / denominator
    num_siapa_percentage = num_siapa / denominator
    num_kenapa_percentage = num_kenapa / denominator
    num_bagaimana_percentage = num_bagaimana / denominator
    num_berapa_percentage = num_berapa / denominator
    num_others_percentage = num_others / denominator
    
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa}")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan}")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana}")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa}")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others}")
    print()
        
    print(f"{str(nama)} memiliki banyak pertanyaan APA sekitar: {num_apa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan DIMANA sekitar: {num_dimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KAPAN sekitar: {num_kapan_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan SIAPA sekitar: {num_siapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan KENAPA sekitar: {num_kenapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BAGAIMANA sekitar: {num_bagaimana_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan BERAPA sekitar: {num_berapa_percentage * 100} %")
    print(f"{str(nama)} memiliki banyak pertanyaan LAINNYA sekitar: {num_others_percentage * 100} %")

In [3]:
all_question = [
    'Apa', 'Apakah', 'apa', 'apakah',
    'Dimana', 'dimana', 'mana',
    'Kapan', 'kapan',
    'Siapa', 'siapa',
    'Mengapa', 'Kenapa', 'mengapa', 'kenapa',
    'Bagaimana', 'bagaimana',
    'Berapa', 'Berapakah', 'berapa', 'berapakah'
]

def pick_a_row(data, question_type, number, all_question=all_question):
    
    cols = ['context', 'question', 'answer']
    target_question_type_df = pd.DataFrame(columns=cols)
    index = []
    
    if question_type == 'Apa': list_question = ['Apa', 'Apakah', 'apa', 'apakah']
    elif question_type == 'Dimana': list_question = ['Dimana', 'dimana', 'mana']
    elif question_type == 'Kapan': list_question = ['Kapan', 'kapan']
    elif question_type == 'Siapa': list_question = ['Siapa', 'siapa']
    elif question_type == 'Kenapa': list_question = ['Mengapa', 'Kenapa', 'mengapa', 'kenapa']
    elif question_type == 'Bagaimana': list_question = ['Bagaimana', 'bagaimana']
    elif question_type == 'Berapa': list_question = ['Berapa', 'Berapakah', 'berapa', 'berapakah']

    for i in range(len(data)):

        current_question = data['question'][i].split()
        
        if question_type == 'Lainnya':
            
            if 'Apa' in current_question: pass
            elif 'Apakah' in current_question: pass
            elif 'apa' in current_question: pass
            elif 'apakah' in current_question: pass
            elif 'Dimana' in current_question: pass
            elif 'dimana' in current_question: pass
            elif 'mana' in current_question: pass
            elif 'Kapan' in current_question: pass
            elif 'kapan' in current_question: pass
            elif 'Siapa' in current_question: pass
            elif 'siapa' in current_question: pass
            elif 'Bagaimana' in current_question: pass
            elif 'bagaimana' in current_question: pass
            elif 'Mengapa' in current_question: pass
            elif 'Kenapa' in current_question: pass
            elif 'mengapa' in current_question: pass
            elif 'kenapa' in current_question: pass
            elif 'Berapa' in current_question: pass
            elif 'Berapakah' in current_question: pass
            elif 'berapa' in current_question: pass
            elif 'berapakah' in current_question: pass

            else: 
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)
            continue

        for j in list_question:
            
            if j in current_question:
                target_question_type_df = target_question_type_df.append({
                                             'context': data["context"][i], 
                                             'question': data['question'][i], 
                                             'answer': data['answer'][i]}, 
                                                                   ignore_index=True)

    target_question_type_df = target_question_type_df.sample(n=number, random_state=42)
    target_question_type_df.sort_index(inplace=True)
    index.append(target_question_type_df.index.values)
    return target_question_type_df, index

In [4]:
def pretty_print(data, index):
    print(data['context'][index])
    print()
    print(data['question'][index])
    print()
    print(data['answer'][index])

# Coba POSTagger

In [5]:
!pip install protobuf==3.20.*

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.


In [6]:
MODEL_NER_NAME = "cahya/xlm-roberta-base-indonesian-NER"
nlp_ner = pipeline(task="ner", model=MODEL_NER_NAME, tokenizer=MODEL_NER_NAME)

In [7]:
ner_result = nlp_ner("Grady Barnett dan tim arkeologinya")

In [8]:
ner_result

[{'entity': 'B-PER',
  'score': 0.9966871,
  'index': 1,
  'word': '▁Grad',
  'start': 0,
  'end': 4},
 {'entity': 'B-PER',
  'score': 0.7089117,
  'index': 2,
  'word': 'y',
  'start': 4,
  'end': 5},
 {'entity': 'I-PER',
  'score': 0.9946517,
  'index': 3,
  'word': '▁Bar',
  'start': 5,
  'end': 9},
 {'entity': 'I-PER',
  'score': 0.9938338,
  'index': 4,
  'word': 'nett',
  'start': 9,
  'end': 13}]

# Bagian IDK-MRC

In [9]:
def assign_answer_types(answer, nlp=nlp_ner):
    
    entity_array = []    
    ner_result = nlp(answer)
    
    for i in ner_result:
        entity = i['entity'][2:]
        entity_array.append(entity)

    return list(set(entity_array))

In [10]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'idk_mrc' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    for j in df_test["qas"][i]:
        if len(j['answers']) != 0:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': j['answers'][0]['text'], 
                                                'answer type': assign_answer_types(answer=j['answers'][0]['text'])     
                                             }, 
                                                           ignore_index=True)
        else:
            new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                                'question': j['question'], 
                                                'answer': "-", 
                                                'answer type': list(set())          
                                             }, 
                                                           ignore_index=True)

test_dataset = Dataset.from_dict(new_df_test)

data_qas_id_idk_mrc = DatasetDict({"test": test_dataset})
data_qas_id_idk_mrc = pd.DataFrame(data_qas_id_idk_mrc['test'])
data_qas_id_idk_mrc

  0%|          | 0/3 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████| 378/378 [02:05<00:00,  3.01it/s]


,context,question,answer,answer type
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan ?,1959,[]
1,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan komputer mikro mulai ditinggalkan?,-,[]
2,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan Amerik...,-,[]
3,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer makro mulai dikembangkan?,-,[]
4,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,Komomene sebuah nama yang berdasarkan pada nam...,[WOA]
...,...,...,...,...
839,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,-,[]
840,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, kota ini ...","[ORG, QTY]"
841,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa vo...",-,[]
842,"Tanpa beasiswa, Ogilvy tidak bisa kuliah di Fe...",Apa alasan Ogilvy tidak bisa kuliah di Fettes ...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...,"[DAT, PER, ORG, LOC]"


In [11]:
def count_answer_type(data):
    num_Person = 0
    num_NORP = 0
    num_Facility = 0
    num_Organization = 0
    num_Geo_Political_Entity = 0
    num_Location = 0
    num_Product = 0
    num_Event = 0
    num_Work_of_Art = 0
    num_Law = 0
    num_Language = 0
    num_Date = 0
    num_Time = 0
    num_Percent = 0
    num_Money = 0
    num_Quantity = 0
    num_Ordinal = 0
    num_Cardinal = 0
    num_null = 0
    num_REG = 0
    denominator = 0

    for answer_type_array in data['answer type']:

        if answer_type_array == []:
            answer_type_array = ['NULL']

        for answer_type in answer_type_array:
            denominator += 1
            if answer_type == 'PER': num_Person += 1
            elif answer_type == 'NOR': num_NORP += 1
            elif answer_type == 'FAC': num_Facility += 1
            elif answer_type == 'ORG': num_Organization += 1
            elif answer_type == 'GPE': num_Geo_Political_Entity += 1
            elif answer_type == 'LOC': num_Location += 1
            elif answer_type == 'PRD': num_Product += 1
            elif answer_type == 'EVT': num_Event += 1
            elif answer_type == 'WOA': num_Work_of_Art += 1
            elif answer_type == 'LAW': num_Law += 1
            elif answer_type == 'LAN': num_Language += 1
            elif answer_type == 'DAT': num_Date += 1
            elif answer_type == 'TIM': num_Time += 1
            elif answer_type == 'PRC': num_Percent += 1
            elif answer_type == 'MON': num_Money += 1
            elif answer_type == 'QTY': num_Quantity += 1
            elif answer_type == 'ORD': num_Ordinal += 1
            elif answer_type == 'CRD': num_Cardinal += 1
            elif answer_type == 'REG': num_REG += 1
            elif answer_type == 'NULL': num_null += 1
            else: print(answer_type)

    print(f"Banyak answer type Person sebanyak: {num_Person}, sekitar {num_Person/denominator * 100} %")
    print(f"Banyak answer type NORP sebanyak: {num_NORP}, sekitar {num_NORP/denominator * 100} %")
    print(f"Banyak answer type Facility sebanyak: {num_Facility}, sekitar {num_Facility/denominator * 100} %")
    print(f"Banyak answer type Organization sebanyak: {num_Organization}, sekitar {num_Organization/denominator * 100} %")
    print(f"Banyak answer type Geo-Political Entity sebanyak: {num_Geo_Political_Entity}, sekitar {num_Geo_Political_Entity/denominator * 100} %")
    print(f"Banyak answer type Location sebanyak: {num_Location}, sekitar {num_Location/denominator * 100} %")
    print(f"Banyak answer type Product sebanyak: {num_Product}, sekitar {num_Product/denominator * 100} %")
    print(f"Banyak answer type Event sebanyak: {num_Event}, sekitar {num_Event/denominator * 100} %")
    print(f"Banyak answer type Work of Art sebanyak: {num_Work_of_Art}, sekitar {num_Work_of_Art/denominator * 100} %")
    print(f"Banyak answer type Law sebanyak: {num_Law}, sekitar {num_Law/denominator * 100} %")
    print(f"Banyak answer type Language sebanyak: {num_Language}, sekitar {num_Language/denominator * 100} %")
    print(f"Banyak answer type Date sebanyak: {num_Date}, sekitar {num_Date/denominator * 100} %")
    print(f"Banyak answer type Time sebanyak: {num_Time}, sekitar {num_Time/denominator * 100} %")
    print(f"Banyak answer type Percent sebanyak: {num_Percent}, sekitar {num_Percent/denominator * 100} %")
    print(f"Banyak answer type Money sebanyak: {num_Money}, sekitar {num_Money/denominator * 100} %")
    print(f"Banyak answer type Quantity sebanyak: {num_Quantity}, sekitar {num_Quantity/denominator * 100} %")
    print(f"Banyak answer type Ordinal sebanyak: {num_Ordinal}, sekitar {num_Ordinal/denominator * 100} %")
    print(f"Banyak answer type Cardinal sebanyak: {num_Cardinal}, sekitar {num_Cardinal/denominator * 100} %")
    print(f"Banyak answer type REG sebanyak: {num_REG}, sekitar {num_REG/denominator * 100} %")
    print(f"Banyak answer type Null sebanyak: {num_null}, sekitar {num_null/denominator * 100} %")

    assert denominator == num_Person + num_NORP + num_Facility + num_Organization + num_Geo_Political_Entity + \
                            num_Location + num_Product + num_Event + num_Work_of_Art + num_Law + \
                            num_Language + num_Date + num_Time + num_Percent + num_Money + \
                            num_Quantity + num_Ordinal + num_Cardinal + num_null + num_REG

In [12]:
count_answer_type(data_qas_id_idk_mrc)

Banyak answer type Person sebanyak: 63, sekitar 6.95364238410596 %
Banyak answer type NORP sebanyak: 9, sekitar 0.9933774834437087 %
Banyak answer type Facility sebanyak: 2, sekitar 0.22075055187637968 %
Banyak answer type Organization sebanyak: 9, sekitar 0.9933774834437087 %
Banyak answer type Geo-Political Entity sebanyak: 60, sekitar 6.622516556291391 %
Banyak answer type Location sebanyak: 25, sekitar 2.759381898454746 %
Banyak answer type Product sebanyak: 30, sekitar 3.3112582781456954 %
Banyak answer type Event sebanyak: 10, sekitar 1.1037527593818985 %
Banyak answer type Work of Art sebanyak: 7, sekitar 0.772626931567329 %
Banyak answer type Law sebanyak: 1, sekitar 0.11037527593818984 %
Banyak answer type Language sebanyak: 2, sekitar 0.22075055187637968 %
Banyak answer type Date sebanyak: 84, sekitar 9.271523178807946 %
Banyak answer type Time sebanyak: 1, sekitar 0.11037527593818984 %
Banyak answer type Percent sebanyak: 2, sekitar 0.22075055187637968 %
Banyak answer type M

In [13]:
question_type(data=data_qas_id_idk_mrc, nama="IDK-MRC Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 14
# DIMANA sebanyak 13
# KAPAN sebanyak 14
# SIAPA sebanyak 13
# KENAPA sebanyak 8
# BAGAIMANA sebanyak 12
# BERAPA sebanyak 13
# LAINNYA sebanyak 13

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 268
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 49
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 173
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 76
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 8
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 12
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 109
IDK-MRC Test memiliki banyak pertanyaan LAINNYA sekitar: 149

IDK-MRC Test memiliki banyak pertanyaan APA sekitar: 31.753554502369667 %
IDK-MRC Test memiliki banyak pertanyaan DIMANA sekitar: 5.80568720379147 %
IDK-MRC Test memiliki banyak pertanyaan KAPAN sekitar: 20.497630331753555 %
IDK-MRC Test memiliki banyak pertanyaan SIAPA sekitar: 9.004739336492891 %
IDK-MRC Test memiliki banyak pertanyaan KENAPA sekitar: 0.9478672985781991 %
IDK-MRC Test memiliki banyak pertanyaan BAGAIMANA sekitar: 1.4218009478672986 %
IDK-MRC Test memiliki banyak pertanyaan BERAPA sekitar: 12.914691943

In [14]:
apa_df_idk_mrc, index_apa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Apa", number=14)
dimana_df_idk_mrc, index_dimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Dimana", number=13)
kapan_df_idk_mrc, index_kapan_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Kapan", number=14)
siapa_df_idk_mrc, index_siapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Siapa", number=13)
kenapa_df_idk_mrc, index_kenapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Kenapa", number=8)
bagaimana_df_idk_mrc, index_bagaimana_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Bagaimana", number=12)
berapa_df_idk_mrc, index_berapa_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Berapa", number=13)
lainnya_df_idk_mrc, index_lainnya_df_idk_mrc = pick_a_row(data=data_qas_id_idk_mrc, question_type="Lainnya", number=14)

In [15]:
data_qas_id_idk_mrc

,context,question,answer,answer type
0,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan ?,1959,[]
1,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan komputer mikro mulai ditinggalkan?,-,[]
2,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer mikro mulai dikembangkan Amerik...,-,[]
3,Ada beberapa inovasi dari GAN yang sedikit nga...,Kapan Komputer makro mulai dikembangkan?,-,[]
4,"Patronim, atau patronimik, adalah sebuah kompo...",Apakah pengertian matronimik?,Komomene sebuah nama yang berdasarkan pada nam...,[WOA]
...,...,...,...,...
839,Umur dewasa dari drummer ganda adalah sekitar ...,Bagaimana cara jangkrik betina hidup?,-,[]
840,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa lu...","Menurut Biro Sensus Amerika Serikat, kota ini ...","[ORG, QTY]"
841,Kota ini terletak di sebuah lembah sungai yang...,"Menurut Biro Sensus Amerika Serikat, berapa vo...",-,[]
842,"Tanpa beasiswa, Ogilvy tidak bisa kuliah di Fe...",Apa alasan Ogilvy tidak bisa kuliah di Fettes ...,Ogilvy tidak bisa kuliah di Fettes atau Oxford...,"[DAT, PER, ORG, LOC]"


In [16]:
#pd.set_option('display.height',1000)
pd.set_option('display.max_rows',500)
pd.set_option('display.max_columns',500)
pd.set_option('display.width',1000)

In [17]:
print(index_apa_df_idk_mrc)
apa_df_idk_mrc

# 22 : AoI
# 30 : AoI
# 67 : MSR
# 79 : WM
# 82 : AoI

# 115 : SSR
# 117 : AoI
# 126 : AoI
# 132 : PP
# 137 : AoI

# 159 : PP
# 184 : PP
# 213 : AoI
# 234 : AoI

reasoning_type = ['AoI', 'AoI', 'MSR', 'WM', 'AoI',
                 'SSR', 'AoI', 'AoI', 'PP', 'AoI',
                 'PP', 'PP', 'AoI', 'AoI']

apa_df_idk_mrc['Reasoning Type'] = reasoning_type
apa_df_idk_mrc

[array([ 22,  30,  67,  79,  82, 115, 117, 126, 132, 137, 159, 184, 213,
       234])]


,context,question,answer,Reasoning Type
22,Pornografi dalam rancangan pertama didefinisik...,Apa yang dimaksud dengan pornoaksi dalam media...,-,AoI
30,Fadjroel Rachman lahir di Banjarmasin pada tan...,apakah pencapaian terakhir Mochamad Fadjroel R...,-,AoI
67,Kelompok masyarakat ini telah menunjukan eksis...,Apakah ajaran yang dipercayai Suku Dayak Hindu...,agama Jawa,MSR
79,Pound Lebanon ( lira; bahasa perancis: livre; ...,Apa nama mata uang Lebanon ?,Pound Lebanon,WM
82,"Som (Uzbek: soʻm dalam Latin, сўм dalam Sirili...",Apa nama mata uang ajikistan?,-,AoI
115,jmpl|200px|Sebuah gereja Kristen berada di bel...,Apa agama mayoritas penduduk korea ?,tidak memiliki agama,SSR
117,jmpl|200px|Sebuah gereja Kristen berada di bel...,Apa agama mayoritas penduduk korea pada tahun ...,-,AoI
126,Entitas adalah sesuatu yang memiliki keberadaa...,Apakah pengertian dari non-entitas?,-,AoI
132,Joseph Pulitzer adalah seorang jurnalis dan pe...,apakah nama penghargaan tertinggi di bidang ju...,Penghargaan Pulitzer,PP
137,Marc Marquez membuat debutnya di kejuaraan pad...,Di tim apakah Marcel MARquez Alenta mendapatka...,-,AoI


In [18]:
print(index_dimana_df_idk_mrc)
dimana_df_idk_mrc

# 4 : AoI
# 12 : AoI
# 13 : AoI
# 17 : AoI
# 19 : AoI

# 25 : AoI
# 26 : MSR
# 27 : AoI
# 31 : AoI
# 34 : WM

# 44 : WM
# 45 : AoI
# 47 : AoI

reasoning_type = ['AoI', 'AoI', 'AoI', 'AoI', 'AoI',
                 'AoI', 'MSR', 'AoI', 'AoI', 'WM',
                 'WM', 'AoI', 'AoI']

dimana_df_idk_mrc['Reasoning Type'] = reasoning_type
dimana_df_idk_mrc

[array([ 4, 12, 13, 17, 19, 25, 26, 27, 31, 34, 44, 45, 47])]


,context,question,answer,Reasoning Type
4,New Orleans (/[invalid input: 'icon']njuː ˈɔːr...,Di mana letak Monroe?,-,AoI
12,"Lahir di Indonesia, Besar di Indonesia menjadi...",Di mana Siauw Giok Tjhan meninggal?,-,AoI
13,"Lahir di Indonesia, Besar di Indonesia menjadi...",Di mana Siauw Giok Tjhan dipenjara?,-,AoI
17,Pelemparan di Praha (English: Defenestrations ...,Di mana Pelemparan di Texas terjadi?,-,AoI
19,Su Dingfang dilahirkan di Wuyi (sekarang Hengs...,Di mana Su Tang lahir?,-,AoI
25,Penghargaan Nobel dianugerahkan setiap tahun k...,Di mana Penghargaan Nobel dilaksanakan?,-,AoI
26,"Tempat kelahirannya adalah Gorinchem, dan ayah...",Dimana Philip Heinrich Christoph Hofman lahir?,Gorinchem,MSR
27,"Tempat kelahirannya adalah Gorinchem, dan ayah...",Di mana Philip Heinrich Christoph Hofman menin...,-,AoI
31,Istana Kensington merupakan sebuah kediaman ke...,Di mana letak Kensington yang berada di bagian...,-,AoI
34,Montenegro (bahasa Montenegro: Црна Гора / Crn...,Dimana letak Montenegro?,Eropa Selatan,WM


In [19]:
print(index_kapan_df_idk_mrc)
kapan_df_idk_mrc

# 15 : AoI
# 38 : AoI
# 41 : AoI
# 42 : WM
# 60 : SSR

# 78 : PP
# 82 : MSR
# 90 : AoI
# 108 : AoI
# 113 : AoI

# 114 : MSR
# 124 : AoI
# 137 : WM
# 162 : MSR

reasoning_type = ['AoI', 'AoI', 'AoI', 'WM', 'SSR',
                 'PP', 'MSR', 'AoI', 'AoI', 'AoI',
                 'MSR', 'AoI', 'WM', 'MSR']

kapan_df_idk_mrc['Reasoning Type'] = reasoning_type
kapan_df_idk_mrc

[array([ 15,  38,  41,  42,  60,  78,  82,  90, 108, 113, 114, 124, 137,
       162])]


,context,question,answer,Reasoning Type
15,Sejarah Gereja Katolik di Indonesia berawal da...,Kapan agama Katolik masuk ke Roma?,-,AoI
38,"Insiden Honnoji (本能寺の変, Honnōji-no-hen) adalah...",Kapan Insiden Hincroft terjadi?,-,AoI
41,"Lahir di Indonesia, Besar di Indonesia menjadi...",Kapan Siauw Giok Tjhan lahir?,-,AoI
42,Pelemparan di Praha (English: Defenestrations ...,Kapan Pelemparan di Praha terjadi?,1419,WM
60,Bär dilahirkan tanggal 21 Maret 1913 di Sommer...,Kapan Oskar-Heinz “Pritzl” Bär pertama kali ik...,1939,SSR
78,"Pieter Willem Botha, (Paul-Roux, Free State, 1...",Kapan Pieter Willem Botha lahir ?,12 Januari 1916,PP
82,"Pada tahun 1900, Hofman berkenalan dengan seor...",Kapan Philip Heinrich Christoph Hofman mulai m...,1900,MSR
90,Sinode Gereja Bethany Indonesia berdiri dan di...,Kapan GBI Bethany dihapus?,-,AoI
108,Sejak Oktober 2011 Petar Segrt adalah pelatih ...,Kapan Jan ditunjuk menjadi pelatih kepala PSM ...,-,AoI
113,Setelah resmi menjadi prajurit TNI Angkatan Ud...,Kapan anak dari Mayor PNB Djalaludin Tantu ber...,-,AoI


In [20]:
print(index_siapa_df_idk_mrc)
siapa_df_idk_mrc

# 0 : AoI
# 4 : AoI
# 10 : AoI
# 18 : AoI
# 28 : AoI

# 35 : AoI
# 45 : AoI
# 47 : AoI
# 50 : AoI
# 53 : MSR

# 65 : MSR
# 67 : WM
# 70 : PP

reasoning_type = ['AoI', 'AoI', 'AoI', 'AoI', 'AoI',
                 'AoI', 'AoI', 'AoI', 'AoI', 'MSR',
                 'MSR', 'WM', 'PP']

siapa_df_idk_mrc['Reasoning Type'] = reasoning_type
siapa_df_idk_mrc

[array([ 0,  4, 10, 18, 28, 35, 45, 47, 50, 53, 65, 67, 70])]


,context,question,answer,Reasoning Type
0,John Logie Baird () adalah penemu yang pertama...,Siapa penemu televisi pada tahun 1871?,-,AoI
4,Muhammad (1405-1415) Ahmad (1415-1425) - menan...,Siapa Sultan terakhir Brunei Darussalam?,-,AoI
10,"Baudouin adalah putra Eustace II, Comte Boulog...",Siapa ibu Baudouin I yang paling lemah lembut?,-,AoI
18,Empat Sehat Lima Sempurna adalah kampanye yang...,Siapa yang terakhir menciptakan ide 4 sehat 5 ...,-,AoI
28,Kongres Pemuda yang pertama ini dilaksanakan d...,Siapa yang memimpin Kongres Pemuda Indonesia II?,-,AoI
35,"Pada tahun 2004, atas masukan dan harapan dari...",Siapa yang bukan pendiri Institut Leimena?,-,AoI
45,Pemimpin lama Republik Sosialis Soviet Turkmen...,Siapa yang bukan presiden Kazakhstan pertama?,-,AoI
47,Eiffel... I'm in Love adalah film drama remaja...,Siapa artis wanita pemeran utama film terbaru?,-,AoI
50,Pada tahun 2000 Arief Rachman sempat aktif seb...,Siapa pembawa acara Dunia Fajar?,-,AoI
53,DailyMotion dibuat oleh Benjamin Bejbaum dan O...,Siapa pendiri Dailymotion ?,Benjamin Bejbaum dan Oliver Poitrey,MSR


In [21]:
print(index_kenapa_df_idk_mrc)
kenapa_df_idk_mrc

# 0 : MSR
# 1 : AoI
# 2 : MSR
# 3 : AoI
# 4 : AoI

# 5 : AoI
# 6 : AoI
# 7 : AoI

reasoning_type = ['MSR', 'AoI', 'MSR', 'AoI', 'AoI',
                 'AoI', 'AoI', 'AoI']

kenapa_df_idk_mrc['Reasoning Type'] = reasoning_type
kenapa_df_idk_mrc

[array([0, 1, 2, 3, 4, 5, 6, 7])]


,context,question,answer,Reasoning Type
0,Hamzah bin Abdul-Muththalib (Arabic: حمزه بن ع...,Kenapa Hamzah bin Abdul-Muththalib dijuluki si...,karena kepahlawanannya saat membela Islam,MSR
1,Hamzah bin Abdul-Muththalib (Arabic: حمزه بن ع...,Kenapa Hamzah bin Abdul-Muththalib tidak dijul...,-,AoI
2,Protista adalah mikroorganisme eukariota yang ...,Mengapa protista tidak dikelompokkan ke dalam ...,"Dari sudut pandang taksonomi, pengelompokan in...",MSR
3,Protista adalah mikroorganisme eukariota yang ...,Mengapa protista dikelompokkan ke dalam satu k...,-,AoI
4,"Iodin atau Iodium ( - ungu), adalah unsur kimi...",Mengapa nomor atom Iodin 53?,-,AoI
5,Penemu lampu lalu lintas adalah Lester Farnswo...,Mengapa sistem sinyal stop dan go dipergunakan...,-,AoI
6,PT Kharisma Starvision Plus atau dengan nama u...,Mengapa PT. Kharisma Starvision Plus didirikan?,-,AoI
7,"Nepal nyaris berbentuk segi empat, dengan panj...",Mengapa daerah Nepal dan Bangladesh menyatu?,-,AoI


In [22]:
print(index_bagaimana_df_idk_mrc)
bagaimana_df_idk_mrc

# 0 : AoI
# 1 : AoI
# 2 : AoI
# 3 : AoI
# 4 : MSR

# 5 : AoI
# 6 : AoI
# 7 : AoI
# 8 : AoI
# 9 : SSR

# 10 : AoI
# 11 : AoI

reasoning_type = ['AoI', 'AoI', 'AoI', 'AoI', 'MSR',
                 'AoI', 'AoI', 'AoI', 'AoI', 'SSR',
                 'AoI', 'AoI']

bagaimana_df_idk_mrc['Reasoning Type'] = reasoning_type
bagaimana_df_idk_mrc

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11])]


,context,question,answer,Reasoning Type
0,"Kembali ke Amerika Serikat, Handler mendesain ...",Bagaimana Barbie diciptakan?,-,AoI
1,Istilah Penyiar Radio ini pertama kali digunak...,Bagaimana musik hip hop menjadi bahasa di dunia?,-,AoI
2,Para pakar studi Biblika memberikan nama yang ...,Bagaimana Kitab Yesaya ditulis?,-,AoI
3,Aquitaine (bahasa Gascon dan Occitan: Aquitàni...,Bagaimana dengan letak Austria?,-,AoI
4,Gerakan Kepanduan adalah sebuah gerakan pembin...,Bagaimana tujuan pendidikan non-formal kepramu...,Tujuan ini dicapai melalui program latihan dan...,MSR
5,Gerakan Kepanduan adalah sebuah gerakan pembin...,Bagaimana tujuan pendidikan formal kepramukaan...,-,AoI
6,Suaka margasatwa Muara Angke (SMMA) adalah seb...,Bagaimana letak Suaka margasatwa Muara Angke?,-,AoI
7,"Dilahirkan dengan nama Ahmad Syahiduddin, pada...",Bagaimana cara Drs. K.H. Ahmad Syahiduddin dil...,-,AoI
8,Vaidišová memulai debutnya dalam tenis profesi...,Bagaimana Nicole Vaidišová mulai menjadi petenis?,-,AoI
9,Pemerintah Pakistan (Urdu: حکومتِ پاکستان‎) ad...,Bagaimana sistem pemerintahan Pakistan ?,republik demokrasi parlementer,SSR


In [23]:
print(index_berapa_df_idk_mrc)
berapa_df_idk_mrc

# 0 : SSR
# 4 : MSR
# 10 : AoI
# 11 : AoI
# 30 : AoI

# 45 : MSR
# 64 : AoI
# 68 : MSR
# 78 : AoI
# 79 : AoI

# 80 : MSR
# 84 : AoI
# 96 : PP

reasoning_type = ['SSR', 'MSR', 'AoI', 'AoI', 'AoI',
                 'MSR', 'AoI', 'MSR', 'AoI', 'AoI',
                 'MSR', 'AoI', 'PP']

berapa_df_idk_mrc['Reasoning Type'] = reasoning_type
berapa_df_idk_mrc

[array([ 0,  4, 10, 11, 30, 45, 64, 68, 78, 79, 80, 84, 96])]


,context,question,answer,Reasoning Type
0,Masjid Wazir Khan (Punjabi dan Urdu: مسجد وزیر...,Berapa lama proses pembangunan Masjid Wazir Khan?,"han dimulai pada tahun 1634 masehi, dan selesa...",SSR
4,Pada 19 Juni semua partai di Jerman harus suda...,Ada berapa kandidat dalam Pemilu Jerman 2017?,51,MSR
10,Jumlah pasukan yang mempertahankan Roma tidakl...,Berapa jumlah pasukan yang menolak Roma?,-,AoI
11,"Terakhir, setelah diumumkan oleh BNPB pada 10 ...",Berapakah jumlah korban jiwa tsunami Sulawesi ...,2.045,AoI
30,"Satu hal yang unik, Megatron mampu membuat uku...",Berapa tinggi Decepticons?,-,AoI
45,Krakatau (atau Rakata) adalah kepulauan vulkan...,berapakah jumlah pulau di kepulauan Krakatau?,empat,MSR
64,Pengambilan gambar utama pada film dimulai di ...,tahun berapakah The Greatest Showman dirilis?,20 Desember 2017,AoI
68,Kepulauan Marshall adalah sebuah negara mariti...,berapakah luas Kepulauan Marshall?,luas perairan 750.000 mil persegi (1.900.000 k...,MSR
78,"Secara geografis, Pulau Enggano berada di wila...",Berapakah luas anak Pulau Enggano?,-,AoI
79,"Secara geografis, Pulau Enggano berada di wila...",Berapakah luas Pulau Enggano pada tahun 2022?,-,AoI


In [24]:
print(index_lainnya_df_idk_mrc)
lainnya_df_idk_mrc

# 9 : WM
# 12 : AoI
# 18 : SSR
# 31 : AoI
# 36 : AoI

# 64 : SSR
# 68 : AoI
# 73 : PP
# 76 : AoI
# 78 : MSR

# 82 : PP
# 109 : PP
# 117 : MSR
# 140 : AoI

reasoning_type = ['WM', 'AoI', 'SSR', 'AoI', 'AoI',
                 'SSR', 'AoI', 'PP', 'AoI', 'MSR',
                 'PP', 'PP', 'MSR', 'AoI']

lainnya_df_idk_mrc['Reasoning Type'] = reasoning_type
lainnya_df_idk_mrc

[array([  9,  12,  18,  31,  36,  64,  68,  73,  76,  78,  82, 109, 117,
       140])]


,context,question,answer,Reasoning Type
9,Bidara diperkirakan memiliki asal usul dari As...,dari manakah asal pohon Bidara ?,Asia Tengah,WM
12,Para Redemptoris yang ditunjuk sebagai misiona...,Siapakah anak Bunda Maria Penolak Abadi?,-,AoI
18,"Dilahirkan sebagai keturunan bangsawan Jawa, S...",Dari manakah asal Mr. Raden Mas Sartono?,Jawa,SSR
31,Suku Sasak adalah suku bangsa yang mendiami pu...,dari manakah asalsuku-bangsa Sasak?,pulau Lombok,AoI
36,Manokwari terletak di pantai utara Daerah Kepa...,Di manakah lokasi Amanokwari?,-,AoI
64,"Saat ini, 2019, hanya ada satu mantan presiden...",siapakah presiden russia pada tahun 2012?,Dmitry Medvedev,SSR
68,Tahun awal perkembangan binaraga di dunia bara...,Siapakah Bapak Binaraga Tradisional?,-,AoI
73,Penemu lampu lalu lintas adalah Lester Farnswo...,siapakah yang menciptakan Lampu lalu lintas ?,Lester Farnsworth Wire,PP
76,"Kubilai Khan (bahasa Mongol: Хубилай хаан), Kh...",Di manakah Kubilai Khan lahir?,-,AoI
78,Genosida atau genosid (Bahasa Inggris: genocid...,dari manakah istilah genosida berasal?,bahasa Yunani,MSR


In [25]:
df_idk_mrc_with_rs = pd.concat([apa_df_idk_mrc, 
                        dimana_df_idk_mrc,
                        kapan_df_idk_mrc,
                        siapa_df_idk_mrc,
                        kenapa_df_idk_mrc,
                        bagaimana_df_idk_mrc,
                        berapa_df_idk_mrc,
                        lainnya_df_idk_mrc])

df_idk_mrc_with_rs['Reasoning Type'].value_counts()

AoI    63
MSR    16
PP      9
WM      7
SSR     6
Name: Reasoning Type, dtype: int64

# Bagian TYDIQA-ID

In [26]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'tydiqa_id' in x.dataset_name)[0].load_dataset()

df_test = pd.DataFrame(data_qas_id['test'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in range(len(df_test['context'])):
    answer_start = df_test['context'][i].index(df_test['label'][i])
    answer_end = answer_start + len(df_test['label'][i])
    new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                        'question': df_test["question"][i], 
                                        'answer': df_test["label"][i],
                                        'answer type': assign_answer_types(answer=df_test["label"][i]) 
                                     }, ignore_index=True)
   
test_dataset = Dataset.from_dict(new_df_test)
data_qas_id_tydiqa_id = DatasetDict({"test": test_dataset})
data_qas_id_tydiqa_id = pd.DataFrame(data_qas_id_tydiqa_id['test'])
data_qas_id_tydiqa_id

  0%|          | 0/3 [00:00<?, ?it/s]

,context,question,answer,answer type
0,Ernest Douwes Dekker wafat dini hari tanggal 2...,dimanakah Dr. Ernest François Eugène Douwes D...,28 Agustus 1950,[DAT]
1,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl,[REG]
2,Sebagai tindak lanjut Atlantic Charter tersebu...,Kapan PBB mulai terbentuk ?,24 Oktober 1945,[DAT]
3,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,Puri Dragsholm,[PER]
4,"Lahir di Sheffield, South Yorkshire, Vardy mem...",Dimana Jamie Richard Vardy lahir?,"Sheffield, South Yorkshire","[GPE, LOC]"
...,...,...,...,...
850,Raden Patah (Jawa: ꦫꦢꦺꦤ꧀ꦦꦠꦃcode: jav promoted ...,Siapa raja Demak pertama ?,Raden Patah,[PER]
851,Laut dalam adalah lapisan terbawah dari lautan...,Berapakah kedalaman laut yang disebut dengan l...,1828 m,[DAT]
852,"Justus Heurnius (lahir di Utrecht, Belanda, 15...",Kapan Justus Heurnius lahir ?,1587,[DAT]
853,Frekuensi suara atau frekuensi audio yaitu get...,Apakah yang dimaksud dengan frekuensi audio?,getaran frekuensi yang terdengar oleh manusia ...,[QTY]


In [27]:
count_answer_type(data_qas_id_tydiqa_id)

Banyak answer type Person sebanyak: 138, sekitar 14.665249734325187 %
Banyak answer type NORP sebanyak: 21, sekitar 2.2316684378320937 %
Banyak answer type Facility sebanyak: 5, sekitar 0.5313496280552604 %
Banyak answer type Organization sebanyak: 18, sekitar 1.9128586609989375 %
Banyak answer type Geo-Political Entity sebanyak: 121, sekitar 12.858660998937301 %
Banyak answer type Location sebanyak: 35, sekitar 3.7194473963868226 %
Banyak answer type Product sebanyak: 52, sekitar 5.526036131774708 %
Banyak answer type Event sebanyak: 3, sekitar 0.3188097768331562 %
Banyak answer type Work of Art sebanyak: 12, sekitar 1.2752391073326248 %
Banyak answer type Law sebanyak: 1, sekitar 0.10626992561105207 %
Banyak answer type Language sebanyak: 8, sekitar 0.8501594048884166 %
Banyak answer type Date sebanyak: 144, sekitar 15.3028692879915 %
Banyak answer type Time sebanyak: 0, sekitar 0.0 %
Banyak answer type Percent sebanyak: 1, sekitar 0.10626992561105207 %
Banyak answer type Money seban

In [28]:
question_type(data=data_qas_id_tydiqa_id, nama="TyDIQA-ID Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 15
# DIMANA sebanyak 14
# KAPAN sebanyak 15
# SIAPA sebanyak 15
# KENAPA sebanyak 6
# BAGAIMANA sebanyak 5
# BERAPA sebanyak 15
# LAINNYA sebanyak 15

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 301
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 34
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 136
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 56
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 6
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 5
TyDIQA-ID Test memiliki banyak pertanyaan BERAPA sekitar: 131
TyDIQA-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 186

TyDIQA-ID Test memiliki banyak pertanyaan APA sekitar: 35.2046783625731 %
TyDIQA-ID Test memiliki banyak pertanyaan DIMANA sekitar: 3.976608187134503 %
TyDIQA-ID Test memiliki banyak pertanyaan KAPAN sekitar: 15.906432748538013 %
TyDIQA-ID Test memiliki banyak pertanyaan SIAPA sekitar: 6.549707602339182 %
TyDIQA-ID Test memiliki banyak pertanyaan KENAPA sekitar: 0.7017543859649122 %
TyDIQA-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 0.5847953216374269 %
TyDIQA-ID Test memiliki banyak pertanyaan 

In [29]:
apa_df_tydiqa_id, index_apa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Apa", number=15)
dimana_df_tydiqa_id, index_dimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Dimana", number=14)
kapan_df_tydiqa_id, index_kapan_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Kapan", number=15)
siapa_df_tydiqa_id, index_siapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Siapa", number=15)
kenapa_df_tydiqa_id, index_kenapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Kenapa", number=6)
bagaimana_df_tydiqa_id, index_bagaimana_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Bagaimana", number=5)
berapa_df_tydiqa_id, index_berapa_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Berapa", number=15)
lainnya_df_tydiqa_id, index_lainnya_df_tydiqa_id = pick_a_row(data=data_qas_id_tydiqa_id, question_type="Lainnya", number=15)

In [30]:
print(index_apa_df_tydiqa_id)
apa_df_tydiqa_id

# 5 : MSR
# 9 : SSR
# 33 : PP
# 45 : MSR
# 60 : MSR

# 116 : PP
# 118 : PP
# 154 : WM
# 164 : PP
# 173 : PP

# 177 : PP
# 198 : SSR
# 228 : MSR
# 230 : PP
# 289 : PP

reasoning_type = ['MSR', 'SSR', 'PP', 'MSR', 'MSR',
                 'PP', 'PP', 'WM', 'PP', 'PP',
                 'PP', 'SSR', 'MSR', 'PP', 'PP']

apa_df_tydiqa_id['Reasoning Type'] = reasoning_type
apa_df_tydiqa_id

[array([  5,   9,  33,  45,  60, 116, 118, 154, 164, 173, 177, 198, 228,
       230, 289])]


,context,question,answer,Reasoning Type
5,Gerakan ini kemudian dilanjutkan oleh generasi...,Apakah tujuan dibentuknya Pita Maha?,untuk menjaga standar mutu artistik yang tingg...,MSR
9,Budaya atau kebudayaan berasal dari bahasa San...,Apakah definisi dari budaya ?,"sebagai hal-hal yang berkaitan dengan budi, da...",SSR
33,Genetika (kata serapan dari bahasa Belanda: ge...,Apakah yang dimaksud dengan ilmu genetika ?,cabang biologi yang mempelajari pewarisan sifa...,PP
45,"Sewaktu masih kecil, ia bertemu tiga pelaut Je...",Apa profesi dari Otokichi ?,pelaut,MSR
60,Negara ini dihuni oleh mayoritas orang Flemish...,Apa bahasa mayoritas yang digunakan penduduk B...,Belanda,MSR
116,Antiseptik atau germisida adalah senyawa kimia...,Apakah yang dimaksud dengan antiseptik?,senyawa kimia yang digunakan untuk membunuh at...,PP
118,Hingga kini apa yang menyebabkan seseorang dap...,Apakah penyebab dari autisme ?,faktor genetik atau keturunan dan faktor lingk...,PP
154,Bahasa Inggris adalah bahasa resmi dari Gambia...,Apa bahasa resmi Republik Gambia?,Inggris,WM
164,Produsen etanol terbesar di dunia pada tahun 2...,negara apakah yang paling banyak memproduksi e...,Amerika Serikat,PP
173,Raksa (nama lama: air raksa) atau merkuri atau...,apa lambang merkuri di tabel periodik?,Hg,PP


In [31]:
print(index_dimana_df_tydiqa_id)
dimana_df_tydiqa_id

# 0 : MSR
# 4 : PP
# 8 : PP
# 9 : SSR
# 12 : AoI

# 15 : MSR
# 16 : AoI
# 17 : WM
# 19 : AoI
# 21 : SSR

# 24 : SSR
# 26 : PP
# 27 : WM
# 32 : WM

reasoning_type = ['MSR', 'PP', 'PP', 'SSR', 'AoI',
                 'MSR', 'AoI', 'WM', 'AoI', 'SSR',
                 'SSR', 'PP', 'WM', 'WM']

dimana_df_tydiqa_id['Reasoning Type'] = reasoning_type
dimana_df_tydiqa_id

[array([ 0,  4,  8,  9, 12, 15, 16, 17, 19, 21, 24, 26, 27, 32])]


,context,question,answer,Reasoning Type
0,"Dia dipenjarakan di Puri Dragsholm, 75 kilomet...",Dimana James Hepburn meninggal?,Puri Dragsholm,MSR
4,"Leonardo da Vinci wafat di Clos Lucé, Perancis...",Di mana Leonardo da Vinci meninggal?,"Clos Lucé, Perancis",PP
8,Pulau Bali adalah bagian dari Kepulauan Sunda ...,Dimana letak geografis Provinsi Bali?,8°25′23″ Lintang Selatan dan 115°14′55″ Bujur ...,PP
9,jmpl|ka|260px Republik Uzbekistan (kadang-kada...,Dimana letak Uzbekistan?,sebagian di Asia Tengah dan Eropa Timur,SSR
12,Sungai Rhein berasal dari Pegunungan Alpen di ...,Dimana letak anak sungai Vorderrhein?,Danau Tuma dekat celah Oberalp dan melewati Ru...,AoI
15,John Lennon adalah seorang musisi rock Inggris...,Dimana John meninggal?,lorong pintu masuk bangunan di mana ia menetap...,MSR
16,"Pada 1888, Stanislavski mendirikan Perhimpunan...",Dimana Konstantin Sergejevitch Stanislavski be...,Teater Maly,AoI
17,Festivali i Këngës adalah sebuah kontes lagu A...,Dimana Festivali i Këngës diadakan?,Albania,WM
19,Seri Formula Satu berakar pada seri grand prix...,Dimana Formula 1 pertama kali diselenggarakan ?,Eropa,AoI
21,"Pada waktu itu, dengan alat-alat yang sederhan...",Dimana pabrik PT Bintang Toedjoe?,Kawasan Industri Pulogadung,SSR


In [32]:
print(index_kapan_df_tydiqa_id)
kapan_df_tydiqa_id

# 12 : PP
# 19 : MSR
# 26 : AoI
# 31 : AoI
# 36 : SSR

# 42 : SSR
# 44 : PP
# 45 : AoI
# 51 : MSR
# 60 : SSR

# 62 : SSR
# 64 : PP
# 73 : PP
# 81 : MSR
# 128 : MSR

reasoning_type = ['MSR', 'MSR', 'AoI', 'AoI', 'SSR',
                 'SSR', 'PP', 'AoI', 'MSR', 'SSR',
                 'SSR', 'PP', 'PP', 'MSR', 'MSR']

kapan_df_tydiqa_id['Reasoning Type'] = reasoning_type
kapan_df_tydiqa_id

[array([ 12,  19,  26,  31,  36,  42,  44,  45,  51,  60,  62,  64,  73,
        81, 128])]


,context,question,answer,Reasoning Type
12,'Paus Benediktus XV' ( Latin: Benedictus ; I...,Kapan Paus Benediktus XV lahir ?,21 November 1854,MSR
19,"Accel World(アクセル・ワールド,Akuseru Wārudo) adalah s...",Kapan serial Accel World pertama kali terbit ?,Februari 2009,MSR
26,Pertama kali Kyuhyun tampil di media sebagai a...,Kapan Cho Kyuhyun mulai menjadi penyanyi?,26 Mei 2006,AoI
31,Pada tahun 1960 Pihak Permesta Menyatakan kese...,Kapan Permesta bubar ?,1961,AoI
36,Upacara Pelantikan Joko Widodo (Jokowi) sebaga...,Kapan Joko Widodo dilantik menjadi presiden ?,20 Oktober 2014,SSR
42,Manga Naruto pertama kali diterbitkan di Jepan...,Kapan Naruto mulai difilmkan ?,3 Oktober 2002,SSR
44,Peperangan era Napoleon adalah serangkaian pep...,Kapan Peperangan Napoleon terjadi ?,1799,PP
45,Beberapa catatan tertua ditulis oleh para agam...,Kapan Cina masuk ke Indonesia pertama kali ?,abad ke-4,AoI
51,"Pemikiran mengenai evolusi, yakni bahwa spesie...",Kapan teori evolusi ditemukan ?,abad ke-19,MSR
60,"Abraham Lincoln, Presiden Amerika Serikat ke-1...",Kapan Abraham Lincoln meninggal?,14 April 1865,SSR


In [33]:
print(index_siapa_df_tydiqa_id)
siapa_df_tydiqa_id

# 0 : AoI
# 3 : PP
# 5 : AoI
# 8 : SSR
# 12 : MSR

# 13 : MSR
# 19 : AoI
# 26 : AoI
# 30 : PP
# 33 : SSR

# 34 : PP
# 36 : PP
# 44 : PP
# 50 : PP
# 54 : PP

reasoning_type = ['AoI', 'PP', 'AoI', 'SSR', 'MSR',
                 'MSR', 'AoI', 'AoI', 'PP', 'SSR',
                 'PP', 'PP', 'PP', 'PP', 'PP']

siapa_df_tydiqa_id['Reasoning Type'] = reasoning_type
siapa_df_tydiqa_id

[array([ 0,  3,  5,  8, 12, 13, 19, 26, 30, 33, 34, 36, 44, 50, 54])]


,context,question,answer,Reasoning Type
0,"Pada tanggal 18 Februari 2008, desain Yoo Kerl...",Siapa arsitek Balai Kota Seoul?,Yoo Kerl,AoI
3,Tian Zhao muda - (Byron Mann) Protagonis keemp...,Siapa yang mengetuai Operation Just Cause?,Amerika Serikat,PP
5,"Miyazaki, yang juga menyutradarai My Neighbor ...",Siapa pemeran utama film Spirited Away?,Chihiro,AoI
8,Gereja Asiria Timur (English: Assyrian Church ...,Siapa pendiri Gereja Asiria Timur?,Santo Tomas Rasul,SSR
12,Animonsta Studios Sdn. Bhd. adalah sebuah peru...,Siapa pendiri Animonsta Studios?,Nizam Razak,MSR
13,Penaklukan Suriah oleh Muslim terjadi pada par...,Di masa pemerintahan siapa Penaklukan Suriah t...,Kalifah Abu Bakar dan Umar bin Khattab,MSR
19,Variasi teorema ini meliputi beberapa atau tak...,Siapa yang menggunakan Teorema monyet takhingga?,Émile Borel dan Arthur Eddington,AoI
26,Serpihan-serpihan tersebut dikumpulkan oleh An...,Siapa yang pertama kali menemukan UFO ?,Grady Barnett dan tim arkeologinya,AoI
30,Arjuna adalah nama seorang tokoh protagonis d...,Siapa itu Arjuna dalam Mahabharata?,nama seorang tokoh protagonis dalam wiracarita...,PP
33,Don't Forget adalah sebuah lagu yang dinyanyik...,Siapa pencipta lirik lagu Don't Forget ?,Lovato bersama dengan Jonas Brothers,SSR


In [34]:
print(index_kenapa_df_tydiqa_id)
kenapa_df_tydiqa_id

# 0 : MSR
# 1 : WM
# 2 : MSR
# 3 : SSR
# 4 : AoI

# 5 : MSR

reasoning_type = ['MSR', 'WM', 'MSR', 'SSR', 'AoI',
                 'MSR']

kenapa_df_tydiqa_id['Reasoning Type'] = reasoning_type
kenapa_df_tydiqa_id

[array([0, 1, 2, 3, 4, 5])]


,context,question,answer,Reasoning Type
0,Aksi tersebut telah digambarkan sebagai mengam...,Mengapa Aksi 9 September 1796 berakhir mengamb...,tidak ada pihak yang bisa mencapai hasil yang ...,MSR
1,Sulaa (juga sering disebut dengan Desa Topa) m...,Kenapa Topa disebut dengan Negeri Sejuta Mata ...,karena memiliki sumber mata air alami sebagai ...,WM
2,"25 Januari pagi, keadaan berlangsung seperti b...",Mengapa KMP Tampomas II mengalami kebakaran?,"kebocoran bahan bakar, dan puntung rokok yang ...",MSR
3,Nicola Sacco () dan Bartolomeo Vanzetti () ada...,Mengapa Nicola Sacco dan Bartolomeo Vanzetti d...,melakukan pembunuhan dan pencurian,SSR
4,Amelia tampaknya ingin mencoba membuat uang da...,Mengapa Amelia Elizabeth Dyer née Hobley membu...,ingin mencoba membuat uang dari bisnis peterna...,AoI
5,Etnografi Dunia Maya adalah adaptasi metode pe...,mengapa kultur dan manusia tak dapat dipisahkan?,Setiap tindakan manusia pasti bermuatan kultur...,MSR


In [35]:
print(index_bagaimana_df_tydiqa_id)
bagaimana_df_tydiqa_id

# 0 : AoI
# 1 : SSR
# 2 : PP
# 3 : AoI
# 4 : PP

reasoning_type = ['AoI', 'SSR', 'PP', 'AoI', 'PP']

bagaimana_df_tydiqa_id['Reasoning Type'] = reasoning_type
bagaimana_df_tydiqa_id

[array([0, 1, 2, 3, 4])]


,context,question,answer,Reasoning Type
0,Pada hewan multiseluler renda kompleks sel pad...,Bagaimana Bryozoa berkembangbiak?,renda kompleks sel pada permukaan individu ind...,AoI
1,BJT (Bipolar Junction Transistor) adalah salah...,Bagaimana Cara kerja BJT?,dua diode yang terminal positif atau negatifny...,SSR
2,Pengobatan TB menggunakan antibiotik untuk mem...,Bagaimana cara mengobati Tuberkulosis?,menggunakan antibiotik untuk membunuh bakterinya,PP
3,Kerja kursi lontar mengikuti bentuk kokpit pes...,Bagaimana bentuk Kursi lontar?,mengikuti bentuk kokpit pesawat tempur yang di...,AoI
4,"Mesir berbentuk republik sejak 18 Juni 1953, M...",Bagaimana sistem pemerintahan di Mesir ?,republik,PP


In [36]:
print(index_berapa_df_tydiqa_id)
berapa_df_tydiqa_id

# 4 : MSR
# 19 : SSR
# 26 : MSR
# 27 : AoI
# 31 : WM

# 40 : PP
# 55 : PP
# 56 : WM
# 64 : PP
# 69 : WM

# 81 : MSR
# 95 : PP
# 97 : PP
# 105 : SSR
# 115 : SSR

reasoning_type = ['MSR', 'SSR', 'MSR', 'AoI', 'WM',
                 'PP', 'PP', 'WM', 'PP', 'WM',
                 'MSR', 'PP', 'PP', 'SSR', 'SSR']

berapa_df_tydiqa_id['Reasoning Type'] = reasoning_type
berapa_df_tydiqa_id

[array([  4,  19,  26,  27,  31,  40,  55,  56,  64,  69,  81,  95,  97,
       105, 115])]


,context,question,answer,Reasoning Type
4,"Afrika Selatan terletak di 29° 00' S, 24° 00' ...",berapakah luas Republik Afrika Selatan?,1.219.912km²,MSR
19,Paskah juga merujuk pada masa di dalam kalende...,Berapa lama perayaan Paskah berlangsung?,empat puluh hari sejak Minggu Paskah (puncak d...,SSR
26,"Dragon Ball(ドラゴンボール,Doragon Bōru) adalah sebua...",Ada Berapa serial Dragon Ball ?,42,MSR
27,Sensus Penduduk Indonesia 2010 (disingkat SP20...,Berapa populasi penduduk Indonesia tahun 2010 ?,juta orang na,AoI
31,Kalimantan Selatan (disingkat Kalsel) adalah s...,berapakah luas Kalimantan Selatan?,"37.530,52km²",WM
40,"Dengan hanya sepuluh anggota tersisa, Super Ju...",berapakah jumlah anggota boyband Super Junior?,sepuluh,PP
55,"Al-Qur'an terdiri atas 114 surah, 30 juz dan 6...",Ada berapakah Juz dalam Al-Quran ?,30,PP
56,"Dalam ilmu kimia, golongan (dikenal juga sebag...",Berapa jumlah golongan dalam tabel periodik?,18,WM
64,Kejatuhan Soeharto adalah peristiwa mundurnya ...,Pada tahun berapa presiden Soeharto dilengserk...,1998,PP
69,Saat ini pelabuhan Sunda Kelapa memiliki luas ...,berapakah luas pelabuhan Sunda kelapa?,daratan 760 hektare serta luas perairan kolam ...,WM


In [37]:
print(index_lainnya_df_tydiqa_id)
lainnya_df_tydiqa_id

# 15 : PP
# 24 : MSR
# 35 : AoI
# 68 : AoI
# 76 : MSR

# 100 : MSR
# 101 : SSR
# 113 : PP
# 118 : AoI
# 119 : MSR

# 152 : PP
# 156 : PP
# 164 : MSR
# 169 : SSR
# 177 : WM

reasoning_type = ['PP', 'MSR', 'AoI', 'AoI', 'MSR',
                 'MSR', 'SSR', 'PP', 'AoI', 'MSR',
                 'PP', 'PP', 'MSR', 'SSR', 'WM']

lainnya_df_tydiqa_id['Reasoning Type'] = reasoning_type
lainnya_df_tydiqa_id

[array([ 15,  24,  35,  68,  76, 100, 101, 113, 118, 119, 152, 156, 164,
       169, 177])]


,context,question,answer,Reasoning Type
15,"Robert Koch lahir di Klausthal, Hanover, Jerma...",Darimana asal Robert Heinrich Herman Koch?,Jerman,PP
24,Carlos III ([Carlos]error: {{lang-xx}}: text h...,siapakah orang tua dari Carlos III?,"Philippe V dari Spanyol, dan putra sulung istr...",MSR
35,"Setelah menjadi Raja Amarta, Puntadewa berusah...",Siapakah watak istri Yudistira alias Dharmawan...,Dewi Kuntulwinanten,AoI
68,Ketua Umum: Megawati Soekarnoputri Ketua Dewan...,Siapakah ketua umum PDI tahun 2018 ?,Megawati Soekarnoputri,AoI
76,Dinasti Han didirikan oleh pemimpin pemberonta...,siapakah kaisar terlama pada Dinasti Han?,Kaisar Wu,MSR
100,"Menurut Gereja Katolik, Yesus secara pribadi m...",Siapakah pemimpin Gereja Katolik Roma pertama ?,Petrus,MSR
101,Bangsa Yunani adalah bangsa yang pertama diken...,siapakah penemu ilmu Geografi ?,"Thales dari Miletus, Herodotus, Eratosthenes, ...",SSR
113,"Lituania, Republik Lituania (bahasa Lituania: ...",Dimanakah letak Lituania ?,Eropa bagian timur laut,PP
118,Sejak itu berkembanglah keturunannya hingga me...,suku arab manakah yang pertama kali datang ke ...,Orang Hadhrami,AoI
119,Teman-temannya menyarankan dia untuk memperlam...,dimanakah Ibnu Sina dimakamkan?,"Hamadan, Iran",MSR


In [38]:
df_tydiqa_id_with_rs = pd.concat([apa_df_tydiqa_id, 
                        dimana_df_tydiqa_id,
                        kapan_df_tydiqa_id,
                        siapa_df_tydiqa_id,
                        kenapa_df_tydiqa_id,
                        bagaimana_df_tydiqa_id,
                        berapa_df_tydiqa_id,
                        lainnya_df_tydiqa_id])

df_tydiqa_id_with_rs['Reasoning Type'].value_counts()

PP     32
MSR    24
SSR    18
AoI    17
WM      9
Name: Reasoning Type, dtype: int64

# Bagian SQUAD-ID

In [39]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_validation = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_val = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_validation['context']))):
    new_df_val = new_df_val.append({'context': df_validation["context"][i], 
                                    'question': df_validation["question"][i], 
                                    'answer': eval(df_validation["answer"][i][0])['text'],
                                    'answer type': assign_answer_types(answer=eval(df_validation["answer"][i][0])['text'])
                                   },
                                ignore_index=True)

validation_dataset = Dataset.from_dict(new_df_val)

data_qas_id_squad_id = DatasetDict({"validation": validation_dataset})
data_qas_id_squad_id = pd.DataFrame(data_qas_id_squad_id['validation'])
data_qas_id_squad_id

  0%|          | 0/2 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████| 11873/11873 [11:41<00:00, 16.92it/s]


,context,question,answer,answer type
0,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Di negara apa Normandia berada?,Perancis,[GPE]
1,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Kapan Normandia di Normandia?,10 dan ke,[ORD]
2,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Dari negara mana asal Norse?,"Denmark, Islandia dan Norwegia",[GPE]
3,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Siapa pemimpin Norse?,Rollo,[PER]
4,Bangsa Normandia (Norman: Nourmands; Prancis: ...,Abad berapa pertama kali Normandia mendapatkan...,abad ke-10,[CRD]
...,...,...,...,...
11868,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapa satuan gaya yang jarang digunakan sama ...,sthène,[PER]
11869,"Gaya pon memiliki padanan metrik, yang lebih j...",Apa yang tidak memiliki mitra metrik?,pound-force,[PRD]
11870,"Gaya pon memiliki padanan metrik, yang lebih j...",Berapakah gaya yang diberikan oleh gravitasi s...,kilogram-force,[PRD]
11871,"Gaya pon memiliki padanan metrik, yang lebih j...",Kekuatan apa yang mengarah ke satuan massa yan...,kilogram-force,[PRD]


In [40]:
count_answer_type(data_qas_id_squad_id)

Banyak answer type Person sebanyak: 1337, sekitar 10.522587753817096 %
Banyak answer type NORP sebanyak: 294, sekitar 2.3138674641901464 %
Banyak answer type Facility sebanyak: 39, sekitar 0.30694160239257046 %
Banyak answer type Organization sebanyak: 474, sekitar 3.7305210136943177 %
Banyak answer type Geo-Political Entity sebanyak: 1066, sekitar 8.389737132063592 %
Banyak answer type Location sebanyak: 564, sekitar 4.438847788446403 %
Banyak answer type Product sebanyak: 1085, sekitar 8.539272784511255 %
Banyak answer type Event sebanyak: 122, sekitar 0.9601762946639383 %
Banyak answer type Work of Art sebanyak: 64, sekitar 0.5036990398237053 %
Banyak answer type Law sebanyak: 98, sekitar 0.7712891547300488 %
Banyak answer type Language sebanyak: 8, sekitar 0.06296237997796317 %
Banyak answer type Date sebanyak: 819, sekitar 6.445773650243979 %
Banyak answer type Time sebanyak: 5, sekitar 0.03935148748622698 %
Banyak answer type Percent sebanyak: 113, sekitar 0.8893436171887297 %
Ba

In [41]:
question_type(data=data_qas_id_squad_id, nama="SQuAD-ID Test")

# Target: 100 anotasi -> 100/8 = 12.5
# APA sebanyak 13
# DIMANA sebanyak 12
# KAPAN sebanyak 12
# SIAPA sebanyak 13
# KENAPA sebanyak 12
# BAGAIMANA sebanyak 12
# BERAPA sebanyak 13
# LAINNYA sebanyak 13

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 6215
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 899
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 692
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 1030
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 191
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 340
SQuAD-ID Test memiliki banyak pertanyaan BERAPA sekitar: 1470
SQuAD-ID Test memiliki banyak pertanyaan LAINNYA sekitar: 1036

SQuAD-ID Test memiliki banyak pertanyaan APA sekitar: 52.345658216120604 %
SQuAD-ID Test memiliki banyak pertanyaan DIMANA sekitar: 7.5718015665796345 %
SQuAD-ID Test memiliki banyak pertanyaan KAPAN sekitar: 5.82835003790112 %
SQuAD-ID Test memiliki banyak pertanyaan SIAPA sekitar: 8.67514528762739 %
SQuAD-ID Test memiliki banyak pertanyaan KENAPA sekitar: 1.6086919902299335 %
SQuAD-ID Test memiliki banyak pertanyaan BAGAIMANA sekitar: 2.863640192032342 %
SQuAD-ID Test memiliki banyak pertanyaan BERAPA

In [42]:
apa_df_squad_id, index_apa_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Apa", number=13)
dimana_df_squad_id, index_dimana_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Dimana", number=12)
kapan_df_squad_id, index_kapan_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Kapan", number=12)
siapa_df_squad_id, index_siapa_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Siapa", number=13)
kenapa_df_squad_id, index_kenapa_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Kenapa", number=12)
bagaimana_df_squad_id, index_bagaimana_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Bagaimana", number=12)
berapa_df_squad_id, index_berapa_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Berapa", number=13)
lainnya_df_squad_id, index_lainnya_df_squad_id = pick_a_row(data=data_qas_id_squad_id, question_type="Lainnya", number=13)

In [43]:
print(index_apa_df_squad_id)
apa_df_squad_id

# 756 : SSR
# 1780 : AoI
# 2223 : SSR
# 2387 : MSR
# 2596 : AoI

# 2913 : SSR
# 3337 : MSR
# 3353 : AoI
# 3703 : AoI
# 3921 : PP

# 5238 : WM
# 5416 : AoI
# 5871 : PP

reasoning_type = ['SSR', 'AoI', 'SSR', 'MSR', 'AoI',
                 'SSR', 'MSR', 'AoI', 'AoI', 'PP',
                 'WM', 'AoI', 'PP']

apa_df_squad_id['Reasoning Type'] = reasoning_type
apa_df_squad_id

[array([ 756, 1780, 2223, 2387, 2596, 2913, 3337, 3353, 3703, 3921, 5238,
       5416, 5871])]


,context,question,answer,Reasoning Type
756,Victoria mengandung banyak daerah yang berbeda...,Apa burung negara bagian Victoria?,helmeted honeyeater,SSR
1780,Piagam Sosial kemudian diadopsi pada tahun 198...,Apa gelar Piagam Sosial yang ditetapkan untuk ...,"Bab Sosial""",AoI
2223,"""Sisi Barat"" Fresno, juga sering disebut ""Sout...",Kelompok spesifik apa yang ditemukan dalam pop...,Hmong atau Laos,SSR
2387,Jaringan packet switching CYCLADES adalah jari...,Konsep apa yang mempengaruhi ARPANET nanti?,jaringan pertama yang membuat penghuni bertang...,MSR
2596,"Pada awal abad ke-20, kemajuan penting dalam i...",Sekarang mungkin untuk mengubah usia relatif l...,absolute ages,AoI
2913,Salah satu implementasi masifnya yang paling a...,Apa definisi Jerman mengadopsi pemerintahan Ko...,Pembangkangan sipil,SSR
3337,"Setelah kemerdekaan Malaysia pada tahun 1957, ...",Jenis sekolah apa yang akan mereka miliki di T...,"Tipe Nasional"".",MSR
3353,"Sekolah swasta sering Anglikan, seperti King's...",Masyarakat Katolik apa di Wanganui yang berafi...,Masyarakat St Pius X,AoI
3703,"Di sisi lain, ketimpangan ekonomi yang lebih t...",Apa tingkat peningkatan wirausaha berdasarkan?,kebutuhan,AoI
3921,Beberapa teori yang dikembangkan pada tahun 19...,Apa yang menurut laporan tahun 2013 tentang Ni...,"meningkat dengan ketimpangan,",PP


In [44]:
print(index_dimana_df_squad_id)
dimana_df_squad_id

# 70 : PP
# 86 : PP
# 96 : SSR
# 209 : WM
# 247 : AoI

# 321 : SSR
# 327 : PP
# 468 : AoI
# 528 : PP
# 532 : PP

# 656 : PP
# 685 : PP

reasoning_type = ['PP', 'PP', 'SSR', 'WM', 'AoI',
                 'SSR', 'PP', 'AoI', 'PP', 'PP',
                 'PP', 'PP']

dimana_df_squad_id['Reasoning Type'] = reasoning_type
dimana_df_squad_id

[array([ 70,  86,  96, 209, 247, 321, 327, 468, 528, 532, 656, 685])]


,context,question,answer,Reasoning Type
70,Sementara BSkyB telah dikecualikan dari menjad...,Dari konsorsium mana BSkyB dikeluarkan?,digital,PP
86,Victoria (disingkat Vic) adalah sebuah negara ...,Di mana di Australia berada Victoria?,tenggara,PP
96,Pegunungan Alpen Victoria di timur laut adalah...,Ke arah mana sistem sungai meluas?,east-west,SSR
209,Meskipun tidak memiliki koneksi historis ke Ti...,Negara mana yang paling bergantung pada minyak...,Jepang,WM
247,"Keempat, pengadilan nasional memiliki tugas un...",Pengadilan mana yang tidak memiliki kewajiban ...,pengadilan nasional,AoI
321,"Cestida (""hewan sabuk"") adalah hewan planktoni...",Di mana mulut terletak di Aboral?,di tengah tepi yang berlawanan dari pita,SSR
327,"Ctenophores dulu dianggap sebagai ""jalan buntu...",Di mana ikan herbivora terlihat memakan zoopla...,Laut Merah,PP
468,Salah satu implementasi masifnya yang paling a...,Di negara mana Revolusi Velvet 2004?,Ukraina,AoI
528,"Pada tahun 1890-an, Universitas Chicago, takut...",Ke sekolah mana U of C setuju untuk menyediaka...,sekolah yang terafiliasi,PP
532,"Setelah 1940-an, gaya Gothic di kampus mulai m...",Setelah dekade mana gaya modern memberi jalan ...,1940-an,PP


In [45]:
print(index_kapan_df_squad_id)
kapan_df_squad_id

# 165 : AoI
# 176 : AoI
# 211 : PP
# 220 : MSR
# 299 : AoI

# 314 : SSR
# 327 : AoI
# 431 : SSR
# 538 : AoI
# 552 : AoI

# 611 : PP
# 674 : AoI

reasoning_type = ['AoI', 'AoI', 'PP', 'MSR', 'AoI',
                 'SSR', 'AoI', 'SSR', 'AoI', 'AoI',
                 'PP', 'AoI']

kapan_df_squad_id['Reasoning Type'] = reasoning_type
kapan_df_squad_id

[array([165, 176, 211, 220, 299, 314, 327, 431, 538, 552, 611, 674])]


,context,question,answer,Reasoning Type
165,"Hak-hak dasar, seperti dalam hak asasi manusia...",Kapan Pengadilan Eropa dapat menegakkan tindakan?,tidak sesuai dengan hak-hak dasar yang diakui ...,AoI
176,Menyusul pemilihan Partai Buruh Inggris untuk ...,Kapan arahan Cuti Orangtua ditolak?,1996,AoI
211,Untuk menghindari gangguan dengan stasiun tele...,Kapan KMJ-TV pertama kali disiarkan?,1 Juni 1953,PP
220,"Penyakit pes, yang disebabkan oleh Yersinia pe...",Kapan kelaparan Cina dimulai?,1331,MSR
299,Beberapa teori yang dikembangkan pada tahun 19...,Kapan teori yang dikembangkan menunjukkan keti...,tahun 1970-an,AoI
314,Mahasiswa di Universitas Chicago mengelola leb...,Kapan tim RSO Nations menduduki peringkat tera...,2013-14 dan 2014-2015,SSR
327,Ketidakstabilan mengganggu tahun-tahun awal pe...,Kapan Li Tan menolak pemberontakan?,1262,AoI
431,Ketika Eropa barat laut perlahan-lahan mulai m...,Kapan pemanasan cepat dimulai dan membantu veg...,13.000 BP,SSR
538,"Dalam fokusnya pada kekhalifahan, partai menga...",Kapan Kekhalifahan Utsmaniyah dipertahankan?,1924,AoI
552,"Dalam karya akademik anglophone, teori tentang...",Kapan istilah imperialisme pertama kali tidak ...,1870-an,AoI


In [46]:
print(index_siapa_df_squad_id)
siapa_df_squad_id

# 168 : AoI
# 231 : SSR
# 280 : WM
# 346 : MSR
# 536 : WM

# 548 : PP
# 644 : AoI
# 650 : AoI
# 660 : PP
# 777 : PP

# 824 : PP
# 971 : SSR
# 978 : PP

reasoning_type = ['AoI', 'SSR', 'WM', 'MSR', 'WM',
                 'PP', 'AoI', 'AoI', 'PP', 'PP',
                 'PP', 'SSR', 'PP']

siapa_df_squad_id['Reasoning Type'] = reasoning_type
siapa_df_squad_id

[array([168, 231, 280, 346, 536, 548, 644, 650, 660, 777, 824, 971, 978])]


,context,question,answer,Reasoning Type
168,"New Rochelle, yang terletak di county Westches...","Siapa yang bernama ""Bauffet's Point"" di Davenp...",Huguenot,AoI
231,Embargo itu tidak seragam di seluruh Eropa. Da...,Siapa yang ingin Israel menarik diri dari perb...,Ted Heath,SSR
280,Orang Eropa pertama yang melakukan perjalanan ...,Siapa orang Eropa pertama yang melakukan perja...,Francisco de Orellana,WM
346,"Dari Italia, penyakit ini menyebar ke barat la...",Dengan siapa berdagang Kerajaan Polandia?,Inggris,MSR
536,Bangunan pertama dari kampus University of Chi...,Siapa yang membantu mendesain Quadrangles Utama?,"Cobb, Shepley, Rutan dan Coolidge, Holabird & ...",WM
548,"Dalam bisnis, alumni terkemuka termasuk CEO Mi...",Siapa yang mendirikan McKinsey & Company?,James O. McKinsey,PP
644,Sistem birokrasi yang diciptakan oleh Kublai K...,Siapa yang ketiga dari penasihat Cina Kublai?,Yao Shu,AoI
650,Sementara keberadaan departemen-departemen pem...,Siapa yang tidak memiliki kekuatan militer nya...,Kementerian Perang,AoI
660,Pada saat yang sama orang-orang Mongol mengimp...,Siapa yang dibawa orang-orang Mongol ke Cina s...,Muslim Asia Tengah,PP
777,Ketua (atau Wakil Ketua) memutuskan siapa yang...,Siapa yang memutuskan siapa yang akan berbicar...,The Presiding Officer,PP


In [47]:
print(index_kenapa_df_squad_id)
kenapa_df_squad_id

# 15 : AoI
# 30 : WM
# 45 : AoI
# 66 : AoI
# 69 : SSR

# 95 : AoI
# 115 : AoI
# 128 : AoI
# 158 : AoI
# 170 : AoI

# 174 : PP
# 182 : SSR

reasoning_type = ['AoI', 'WM', 'AoI', 'AoI', 'SSR',
                 'AoI', 'AoI', 'AoI', 'AoI', 'AoI',
                 'PP', 'SSR']

kenapa_df_squad_id['Reasoning Type'] = reasoning_type
kenapa_df_squad_id

[array([ 15,  30,  45,  66,  69,  95, 115, 128, 158, 170, 174, 182])]


,context,question,answer,Reasoning Type
15,"Pada 15 Agustus 1971, Amerika Serikat secara s...",Mengapa minyak mulai dihargai dalam hal emas?,"harga minyak dalam dolar, pendapatan riil prod...",AoI
30,"""Kebebasan untuk menyediakan layanan"" di bawah...",Mengapa pengacara Belanda yang pindah ke Belgi...,karena hukum Belanda mengatakan hanya orang ya...,WM
45,"Dalam mode tanpa koneksi, setiap paket menyert...",Mengapa ada kebutuhan akan jalur khusus?,untuk membantu paket menemukan jalannya ke tuj...,AoI
66,Prinsip suksesi fauna didasarkan pada penampil...,Mengapa mudah menggunakan ketidakhadiran organ...,fosil dapat ditemukan secara global pada saat ...,AoI
69,James Hutton sering dipandang sebagai ahli geo...,Mengapa Royal Society menganggap bumi lebih tu...,untuk memungkinkan cukup waktu bagi gunung-gun...,SSR
95,Beberapa terdakwa pembangkangan sipil memilih ...,Mengapa memberikan pidato yang menantang terka...,kurangnya penyesalan,AoI
115,Sebuah studi oleh World Institute for Developm...,Mengapa Oxfam dan Credit Suisse percaya bahwa ...,pengalihan,AoI
128,"Ketika kemampuan seseorang diturunkan, mereka ...",Mengapa kekerasan yang merajalela tidak akan m...,karena takut akan nyawa mereka,AoI
158,"Sejak 7500 tahun yang lalu, sebuah situasi den...",Mengapa sedimentasi alami oleh Rhine mengimban...,Tingkat kenaikan permukaan laut,AoI
170,Gerakan-gerakan Islam seperti Ikhwanul Muslimi...,Mengapa Ikhwanul Muslimin tidak memfasilitasi ...,"menghindari mahar yang mahal tuntutan, bantuan...",AoI


In [48]:
print(index_bagaimana_df_squad_id)
bagaimana_df_squad_id

# 15 : AoI
# 33 : MSR
# 39 : SSR
# 57 : AoI
# 76 : MSR

# 119 : AoI
# 126 : AoI
# 163 : WM
# 182 : AoI
# 258 : AoI

# 267 : SSR
# 322 : AoI

reasoning_type = ['AoI', 'MSR', 'SSR', 'AoI', 'MSR',
                 'AoI', 'AoI', 'WM', 'AoI', 'AoI',
                 'SSR', 'AoI']

bagaimana_df_squad_id['Reasoning Type'] = reasoning_type
bagaimana_df_squad_id

[array([ 15,  33,  39,  57,  76, 119, 126, 163, 182, 258, 267, 322])]


,context,question,answer,Reasoning Type
15,"Namun, beberapa masalah komputasi lebih mudah ...",Apa yang banyak hubungannya dengan bagaimana k...,Mesin Turing non-deterministik,AoI
33,Gereja Katolik di Prancis dan banyak anggotany...,Bagaimana Huguenot membela diri?,milisi sendiri,MSR
39,"Pada tahun 1840-an dan 50-an, ada upaya untuk ...",Bagaimana lap disediakan oleh tumpang tindih p...,memperpanjang permukaan gosokan katup,SSR
57,"Ini berkontribusi pada ""Kejutan Minyak"". Setel...","Sampai kejutan minyak, bagaimana harga emas?",stabil dibandingkan mata uang dan komoditas la...,AoI
76,Sebagian besar spesies adalah hermafrodit — se...,Bagaimana cara plankton berkembang biak?,"dapat menghasilkan telur dan sperma, artinya d...",MSR
119,Baran mengembangkan konsep switching blok pesa...,Bagaimana forward switching tercapai?,paket,AoI
126,"Dalam mode tanpa koneksi, setiap paket menyert...",Jika paket bepergian melalui rute yang berbeda...,Setiap paket dikirim dan dapat pergi melalui r...,AoI
163,Di sebagian besar yurisdiksi (seperti Amerika ...,Bagaimana apoteker diatur di sebagian besar yu...,terpisah dari dokter,WM
182,Klasifikasi Industri Standar dan Sistem Klasif...,Bagaimana perbedaan dalam rekayasa diakui?,dibagi menjadi tiga subsektor,AoI
258,"Secara politis, sistem pemerintahan yang dicip...",Bagaimana perlakuan yang sama antara orang Cin...,kolonial,AoI


In [49]:
print(index_berapa_df_squad_id)
berapa_df_squad_id

# 49 : AoI
# 67 : AoI
# 76 : AoI
# 123 : AoI
# 124 : PP

# 297 : PP
# 432 : PP
# 529 : SSR
# 628 : MSR
# 937 : MSR

# 1031 : SSR
# 1415 : SSR
# 1473 : SSR

reasoning_type = ['AoI', 'AoI', 'AoI', 'AoI', 'PP',
                 'PP', 'PP', 'SSR', 'MSR', 'MSR',
                 'SSR', 'SSR', 'SSR']

berapa_df_squad_id['Reasoning Type'] = reasoning_type
berapa_df_squad_id

[array([  49,   67,   76,  123,  124,  297,  432,  529,  628,  937, 1031,
       1415, 1473])]


,context,question,answer,Reasoning Type
49,California Selatan terdiri dari satu Area Stat...,Berapa populasi di Greater Los Angeles Area?,17.786.419,AoI
67,Setelah pertempuran hukum yang panjang dengan ...,Sky kehilangan berapa banyak paket?,empat,AoI
76,Sky UK Limited (sebelumnya British Sky Broadca...,Berapa banyak pelanggan yang kehilangan Sky UK...,11 juta,AoI
123,Lebih dari 26.000 kilometer persegi (10.000 mi...,Berapa ton kentang yang dihasilkan Victoria?,270.000,AoI
124,Mallee dan Wimmera atas adalah daerah terhanga...,Berapa suhu bulanan tertinggi Victoria?,"48,8 ° C",PP
297,Lokomotif uap kereta api skala penuh pertama d...,Pada tanggal berapa perjalanan kereta api pert...,21 Februari 1804,PP
432,"Untuk membantu mengurangi konsumsi, pada tahun...",Berapa kecepatan yang Undang-Undang Penunjukan...,55 mph,PP
529,Orang Eropa pertama yang melakukan perjalanan ...,Film dokumenter BBC menyajikan bukti bahwa Ama...,11.000 tahun,SSR
628,"Di bagian timur laut Fresno, Woodward Park did...",Berapa banyak kursi yang dimiliki ampiteater d...,2.500,MSR
937,"Harvard memiliki beberapa fasilitas atletik, s...",Berapa banyak ruang berat di Malkin Athletic C...,tiga ruang berat,MSR


In [50]:
pretty_print(lainnya_df_squad_id, 999)

Mendorong benda pada permukaan gesekan dapat mengakibatkan situasi di mana objek tidak bergerak karena gaya yang diberikan ditentang oleh gesekan statis, dihasilkan antara objek dan permukaan meja. Untuk situasi tanpa gerakan, gaya gesek statis persis menyeimbangkan gaya yang diberikan sehingga tidak ada akselerasi. Gesekan statis bertambah atau berkurang sebagai respons terhadap gaya yang diberikan hingga batas atas yang ditentukan oleh karakteristik kontak antara permukaan dan objek.

Menarik benda pada permukaan gesekan dapat menghasilkan apa?

situasi di mana objek tidak bergerak


In [51]:
print(index_lainnya_df_squad_id)
lainnya_df_squad_id

# 101 : AoI
# 109 : AoI
# 198 : PP
# 458 : WM
# 519 : AoI

# 526 : MSR
# 541 : SSR
# 588 : SSR
# 622 : SSR
# 649 : MSR

# 673 : SSR
# 718 : AoI
# 999 : AoI

reasoning_type = ['AoI', 'AoI', 'PP', 'WM', 'AoI',
                 'MSR', 'SSR', 'SSR', 'SSR', 'MSR',
                 'SSR', 'AoI', 'AoI']

lainnya_df_squad_id['Reasoning Type'] = reasoning_type
lainnya_df_squad_id

[array([101, 109, 198, 458, 519, 526, 541, 588, 622, 649, 673, 718, 999])]


,context,question,answer,Reasoning Type
101,Perang agama baru pada tahun 1620-an menyebabk...,Jenis bahasa Prancis manakah yang mengeluarkan...,Louis XIV,AoI
109,Sejumlah Huguenot menjabat sebagai walikota di...,Distrik Huguenot dinamai di Cork pada tahun be...,1696,AoI
198,Cabang yudisial UE telah memainkan peran penti...,Di bawah pengadilan manakah sebagian besar huk...,pengadilan negara anggota,PP
458,Beberapa teori pembangkangan sipil berpendapat...,Beberapa teori berpendapat bahwa pembangkangan...,entitas pemerintah,WM
519,Didirikan awalnya oleh badan legislatif Massac...,Siapakah Presiden Radcliffe dari tahun 1869-1909?,Charles W. Eliot,AoI
526,Harvard memiliki dana abadi universitas terbes...,Seberapa besar kerugian yang diderita dalam kr...,kerugian sekitar 30%,MSR
541,Suhu rata-rata bulanan berkisar dari sekitar 5...,Cara saya bulan paling dingin di Jacksonville?,Januari,SSR
588,"Pada tahun 1890-an, Universitas Chicago, takut...",Universitas setuju untuk memberikan gelar kepa...,nilai A selama empat tahun,SSR
622,Staf pengajar saat ini termasuk antropolog Mar...,Hank Paulson adalah mantan Ketua dan CEO dari ...,Goldman Sachs,SSR
649,"Sel pembunuh alami, atau sel NK, adalah kompon...",Sel pembunuh alami mengenali sel yang harus di...,missing self,MSR


In [52]:
df_squad_id_with_rs = pd.concat([apa_df_squad_id, 
                        dimana_df_squad_id,
                        kapan_df_squad_id,
                        siapa_df_squad_id,
                        kenapa_df_squad_id,
                        bagaimana_df_squad_id,
                        berapa_df_squad_id,
                        lainnya_df_squad_id])

df_squad_id_with_rs['Reasoning Type'].value_counts()

AoI    41
SSR    21
PP     21
MSR    10
WM      7
Name: Reasoning Type, dtype: int64

In [53]:
data_qas_id_idk_mrc.to_csv("idk_mrc.csv")
data_qas_id_tydiqa_id.to_csv("tydiqa_id.csv")
#data_qas_id_squad_id.to_excel("squad_id.xlsx")

# Re-splitting SQuAD-ID

In [ ]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas_id['train'])

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                    'question': df_train["question"][i], 
                                    'answer': eval(df_train["answer"][i][0])['text']},
                                ignore_index=True)

train_dataset = Dataset.from_dict(new_df_train)

data_qas_id_squad_id = DatasetDict({"train": train_dataset})
data_qas_id_squad_id = pd.DataFrame(data_qas_id_squad_id['train'])
data_qas_id_squad_id

  0%|          | 0/2 [00:00<?, ?it/s]

 78%|█████████████████████████████████████████████████████████▌                | 101351/130319 [06:46<03:19, 145.05it/s]

In [ ]:
train = data_qas_id_squad_id[:-11874]
train

In [ ]:
val = data_qas_id_squad_id[-11874:]
val

In [ ]:
question_type(data=train, nama="SQuAD-ID Train")

In [ ]:
question_type(data=val, nama="SQuAD-ID Validation")

In [ ]:
conhelps = NusantaraConfigHelper()
data_qas_id = conhelps.filtered(lambda x: 'squad_id' in x.dataset_name)[0].load_dataset()

df_train = pd.DataFrame(data_qas_id['train'])
df_test = pd.DataFrame(data_qas_id['validation'])

cols = ['context', 'question', 'answer']
new_df_test = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_test['context']))):
    new_df_test = new_df_test.append({'context': df_test["context"][i], 
                                    'question': df_test["question"][i], 
                                    'answer': {"text": eval(df_test["answer"][i][0])['text'], 
                                    "answer_start": eval(df_test["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_test["answer"][i][0])['answer_end']}}, 
                                ignore_index=True)

cols = ['context', 'question', 'answer']
new_df_train = pd.DataFrame(columns=cols)

for i in tqdm(range(len(df_train['context']))):
    new_df_train = new_df_train.append({'context': df_train["context"][i], 
                                    'question': df_train["question"][i], 
                                    'answer': {"text": eval(df_train["answer"][i][0])['text'], 
                                    "answer_start": eval(df_train["answer"][i][0])['answer_start'], 
                                    "answer_end": eval(df_train["answer"][i][0])['answer_end']}}, 
                                ignore_index=True)

train_final_df = new_df_train[:-11874]
validation_final_df = new_df_train[-11874:]

train_dataset = Dataset.from_dict(train_final_df)
validation_dataset = Dataset.from_dict(validation_final_df)
test_dataset = Dataset.from_dict(df_test)

data_qas_id = DatasetDict({"train": train_dataset, "validation": validation_dataset, "test": test_dataset})
data_qas_id

In [ ]:
train = 118445
validation = 11874
test = 11873

denominator = train + validation + test

print(f"Train percentage: {train / denominator * 100}")
print(f"Validation percentage: {validation / denominator * 100}")
print(f"Test percentage: {test / denominator * 100}")